In [1]:
pip install opencv-python numpy mediapipe

Note: you may need to restart the kernel to use updated packages.


In [9]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# Initialize the canvas
canvas = np.zeros((480, 640, 3), dtype=np.uint8)

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Previous coordinates of the drawing finger
prev_x, prev_y = 0, 0

# Drawing settings
draw_color = (255, 255, 255)  # White color
thickness = 5
drawing_mode = True  # True: drawing, False: not drawing

# Define colors
colors = {
    'white': (255, 255, 255),
    'red': (0, 0, 255),
    'green': (0, 255, 0),
    'blue': (255, 0, 0),
    'yellow': (0, 255, 255)
}
color_keys = list(colors.keys())  # List of color names
color_index = 0  # Index to cycle through colors

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)  # Mirror the frame
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame with MediaPipe Hands
    results = hands.process(rgb_frame)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Get the coordinates of the index finger tip
            index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_finger_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
            
            x, y = int(index_finger_tip.x * frame.shape[1]), int(index_finger_tip.y * frame.shape[0])
            
            # Calculate the distance between the index finger tip and thumb tip
            thumb_x, thumb_y = int(thumb_tip.x * frame.shape[1]), int(thumb_tip.y * frame.shape[0])
            distance = np.sqrt((x - thumb_x)**2 + (y - thumb_y)**2)
            
            # Drawing mode or erasing mode
            if distance < 40:  # Distance threshold for erasing
                cv2.circle(frame, (x, y), thickness, (0, 0, 0), cv2.FILLED)
                cv2.circle(canvas, (x, y), thickness, (0, 0, 0), cv2.FILLED)
            elif drawing_mode:
                # Draw on the canvas
                if prev_x != 0 and prev_y != 0:
                    cv2.line(canvas, (prev_x, prev_y), (x, y), draw_color, thickness)
                
            prev_x, prev_y = x, y
            
            # Draw hand landmarks on the frame
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    else:
        # Reset previous coordinates when hand is not detected
        prev_x, prev_y = 0, 0
    
    # Combine the canvas with the frame
    frame = cv2.addWeighted(frame, 1, canvas, 0.5, 0)
    
    # Display the result
    cv2.putText(frame, f'Color: {color_keys[color_index]}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, draw_color, 2)
    cv2.putText(frame, f'Thickness: {thickness}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, draw_color, 2)
    cv2.imshow("Air Canvas", frame)
    
    # Keyboard controls
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('c'):
        canvas = np.zeros((480, 640, 3), dtype=np.uint8)
    elif key == ord('s'):
        cv2.imwrite("canvas.png", canvas)
        print("Canvas saved as canvas.png")
    elif key == ord('m'):
        drawing_mode = not drawing_mode
        mode_text = "Drawing" if drawing_mode else "Not Drawing"
        print(f"Mode: {mode_text}")
    elif key == ord('r'):
        color_index = (color_index + 1) % len(colors)
        draw_color = colors[color_keys[color_index]]
        print(f"Changed color to: {color_keys[color_index]}")
    elif key == ord('+'):
        thickness += 1
    elif key == ord('-') and thickness > 1:
        thickness -= 1

cap.release()
cv2.destroyAllWindows()


Mode: Not Drawing
Mode: Drawing
Changed color to: red
Changed color to: green
Changed color to: blue
